# Household Location Choice model workflow

Max Gardner, Feb 2018  
Python 3.6, intended to be backward compatible with 2.7

In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
from patsy import dmatrix

In [2]:
# Standard to run UrbanSim from the root level of the project directory

import os; os.chdir('../')

In [3]:
import modelmanager as mm
from modelmanager.models import RegressionStep
from modelmanager.models import MNLDiscreteChoiceStep
import orca

## Bootstrap Orca with some legacy registrations

This exercise starts from a point where data is already registered in Orca. For now, we're using a small San Diego dataset that Paul Sohn put together (see [urbansim_parcels](https://github.com/urbansim/urbansim_parcels)). 

Importing 'datasources.py' and 'models.py' from the 'legacy' directory registers a handful of Orca objects.

In [4]:
import legacy_datasources
import legacy_models

## Explore the Orca registrations

In [5]:
orca.list_tables()

['households', 'buildings', 'parcels', 'jobs']

In [6]:
orca.list_columns()

[('households', 'node_id'), ('buildings', 'node_id'), ('jobs', 'node_id')]

In [7]:
orca.list_broadcasts()

[('parcels', 'buildings'),
 ('buildings', 'households'),
 ('buildings', 'jobs'),
 ('nodes', 'buildings')]

In [8]:
orca.list_injectables()

['settings', 'store', 'net_store']

In [9]:
orca.list_steps()

['build_networks', 'neighborhood_vars']

## Explore the data

Orca doesn't execute code to load the registered objects until it needs to.

(Note that there's a problem in the data: only about 2,000 buildings have a 'node_id' linking them to the network aggregations table. This is going to affect the network calculations, and also limit our ability to estimate models.)

In [10]:
orca.get_table('buildings').to_frame().describe()

,parcel_id,development_type_id,improvement_value,residential_units,non_residential_sqft,stories,year_built,residential_sqft,res_price_per_sqft,node_id
count,5.917200e+04,59172.000000,5.917200e+04,59172.000000,5.917200e+04,59172.000000,59172.000000,5.917200e+04,59172.000000,2182.000000
mean,1.824953e+06,19.335733,5.938045e+05,3.662661,8.495205e+03,1.081170,1979.705486,1.303178e+04,214.760125,44284.095325
std,2.138605e+06,0.663925,2.823604e+06,13.734032,4.363458e+04,0.381794,20.675804,4.544811e+04,317.965487,4104.251504
min,2.900000e+01,19.000000,0.000000e+00,1.000000,0.000000e+00,1.000000,1913.000000,5.000000e+01,0.000220,36360.000000
25%,2.894475e+05,19.000000,1.071312e+05,1.000000,0.000000e+00,1.000000,1965.000000,1.472000e+03,133.098672,41131.000000
50%,5.873640e+05,19.000000,1.750000e+05,1.000000,0.000000e+00,1.000000,1981.000000,2.122000e+03,191.676955,43936.500000
75%,5.032626e+06,19.000000,3.022590e+05,1.000000,0.000000e+00,1.000000,1998.000000,3.601000e+03,258.776213,47438.000000
max,5.293696e+06,21.000000,1.800000e+08,679.000000,1.197079e+06,29.000000,2012.000000,1.197079e+06,21770.833333,52513.000000


In [11]:
print(len(orca.get_table('buildings').local_columns))  # native columns only
print(len(orca.get_table('buildings').to_frame().columns))  # native plus virtual

10
11


## Generate accessibility measures for the choice model

The network accessibility metrics are not stored on disk; for now we'll generate them using legacy code.

In [12]:
orca.run(['build_networks'])

Running step 'build_networks'
Time to execute step 'build_networks': 0.22 s
Total time to execute iteration 1 with iteration value None: 0.22 s


In [13]:
%%capture
orca.run(['neighborhood_vars'])

In [14]:
orca.list_tables()

['households', 'buildings', 'parcels', 'jobs', 'nodes']

In [15]:
print(orca.get_table('buildings').to_frame().columns.tolist())

['parcel_id', 'development_type_id', 'improvement_value', 'residential_units', 'non_residential_sqft', 'stories', 'year_built', 'residential_sqft', 'note', 'res_price_per_sqft', 'node_id']


## Display all the registered data columns

In [16]:
for table_name in orca.list_tables():
    print(table_name.upper())
    print(orca.get_table(table_name).to_frame().columns.tolist())
    print()

HOUSEHOLDS
['building_id', 'tenure', 'persons', 'workers', 'age_of_head', 'income', 'children', 'race_id', 'cars', 'base_luz', 'segmentation_col', 'node_id']

BUILDINGS
['parcel_id', 'development_type_id', 'improvement_value', 'residential_units', 'non_residential_sqft', 'stories', 'year_built', 'residential_sqft', 'note', 'res_price_per_sqft', 'node_id']

PARCELS
['distance_to_coast', 'msa_id', 'distance_to_freeway', 'development_type_id', 'x', 'distance_to_transit', 'proportion_undevelopable', 'distance_to_park', 'taz_id', 'tax_exempt', 'parcel_acres', 'county_id', 'distance_to_school', 'distance_to_onramp', 'acres', 'mgra_id', 'land_value', 'node_id', 'luz_id', 'y', 'zoning_id']

JOBS
['sector_id', 'building_id', 'node_id']

NODES
['ave_parcel_size', 'jobs_1500m', 'jobs_800m', 'jobs_400m', 'ave_income', 'ave_age_of_head_1500m', 'ave_children_1500m', 'ave_year_built_1500m', 'population_400m', 'jobs_3000m', 'households_3000m', 'residential_units_3000m', 'residential_units_1500m', 'res

In [17]:
# These are the tables with direct relational links

orca.list_broadcasts()

[('parcels', 'buildings'),
 ('buildings', 'households'),
 ('buildings', 'jobs'),
 ('nodes', 'buildings')]

# Estimate a choice model

The basic idea of the parcel template is that we create model steps by _passing arguments to classes_ rather than by writing Python functions and giving them Orca decorators, as we would for a fully custom model.

Much of the functionality for this is already built into UrbanSim and Orca, we'll just need to extend things here and there.

This demo uses a new MNLDiscreteChoice() class that provides a full model development workflow: estimating a model, registering it with Orca, saving it for future use.

### Specify parameters and pass them to a model object

In [22]:
# Specify the model expression and names of tables to draw data from (the first table
# is the primary one; additional tables must be able to merge onto it unambiguously)

choosers = 'households'
alternatives=['buildings', 'nodes']
current_choice = 'building_id'

model_expression = ("res_price_per_sqft + jobs_1500m")


# Give the prospective model step some tags, and a name if desired

name = 'hlcm1'
tags = ['hlcm', 'max', '201802']


# For prediction, specify destination column (if different from the dependent variable
# used for estimation), and how to reverse the left-hand-side transformation

out_fname = 'fitted_hlcm'

In [23]:
# Generate a new column to store the fitted household location id
zeros = np.repeat(0.0, len(orca.get_table(choosers)))
orca.get_table('households').update_col('fitted_hlcm', zeros)

In [24]:
# Create the model object
model = MNLDiscreteChoiceStep(model_expression, choosers=choosers, alternatives=alternatives,
                              sample_size=50, name=name, tags=tags, out_fname=out_fname)

### Fit the model

In [26]:
model.fit(choosers, alternatives, current_choice)

/home/mgardner/src/urbansim/urbansim/urbanchoice/interaction.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  alts_sample['join_index'] = np.repeat(choosers.index.values, SAMPLE_SIZE)


Null Log-liklihood: -39898.723
Log-liklihood at convergence: -34565.504
Log-liklihood Ratio: 0.134

+--------------------+-------------+------------+----------+
| Component          | Coefficient | Std. Error | T-Score  |
+--------------------+-------------+------------+----------+
| res_price_per_sqft |    -0.007   |   0.000    | -108.228 |
| jobs_1500m         |    0.354    |   0.008    |  43.293  |
+--------------------+-------------+------------+----------+
None


### If we like it, register it as an Orca step

In [27]:
model.register()

In [28]:
orca.list_steps()

['build_networks', 'neighborhood_vars', 'hlcm1']

### Run the Orca step

In [29]:
orca.run(['hlcm1'])

Running step 'hlcm1'


/home/mgardner/src/urbansim/urbansim/urbanchoice/interaction.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  alts_sample['join_index'] = np.repeat(choosers.index.values, SAMPLE_SIZE)


Predicted 58671 values
Time to execute step 'hlcm1': 239.88 s
Total time to execute iteration 1 with iteration value None: 239.88 s


### Check the fitted values

In [31]:
df = orca.get_table(choosers).to_frame(['building_id', 'fitted_hlcm'])
df.loc[df.fitted_hlcm > 0].describe()

,building_id,fitted_hlcm
count,58671.000000,58671.000000
mean,370371.030339,369661.477953
std,79639.958079,80389.698114
min,5120.000000,5345.000000
25%,352274.500000,351548.000000
50%,363553.000000,361720.000000
75%,380838.500000,372950.000000
max,679716.000000,679613.000000


### BONUS

Running "model.register()" also registered the step with the new ModelManager extention, which saves it to disk so that it can be automatically re-loaded in the future.

The "test" model steps here were estimated earlier and loaded from disk. They're fully functional: we can run them in Orca, inspect the estimation results, etc.

In [33]:
mm.list_steps()

[{'name': 'hlcm1',
  'tags': ['hlcm', 'max', '201802'],
  'type': 'MNLDiscreteChoiceStep'}]

In [34]:
rs = mm.get_step('hlcm1')
type(rs)

modelmanager.models.dcm.MNLDiscreteChoiceStep

In [35]:
rs.model_expression

'res_price_per_sqft + jobs_1500m'

In [36]:
rs.model.report_fit()

Null Log-liklihood: -39898.723
Log-liklihood at convergence: -34565.504
Log-liklihood Ratio: 0.134

+--------------------+-------------+------------+----------+
| Component          | Coefficient | Std. Error | T-Score  |
+--------------------+-------------+------------+----------+
| jobs_1500m         |    0.354    |   0.008    |  43.293  |
| res_price_per_sqft |    -0.007   |   0.000    | -108.228 |
+--------------------+-------------+------------+----------+
